In [2]:
import math
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

本项目用来进行机器翻译的任务，实现德语到英语的序列转化

# 1.准备数据集



In [3]:
sentences = [
        # enc_input模型输入           dec_input 模型解码出的结果        dec_output，标签，真实值
        ['ich mochte ein bier P', 'S i want a beer .', 'i want a beer . E'],
        ['ich mochte ein cola P', 'S i want a coke .', 'i want a coke . E']
]# 一行是一个样本
# P表示长度不够时填充的序列
# S表示解码输入开始的符号
# E: 表示解码输出开始的符号
src_vocab = {'P' : 0, 'ich' : 1, 'mochte' : 2, 'ein' : 3, 'bier' : 4, 'cola' : 5}# 手动编码，单词与索引之间映射的关系，这里是德语词表
src_vocab_size = len(src_vocab)# 获取输德语词表长度

tgt_vocab = {'P' : 0, 'i' : 1, 'want' : 2, 'a' : 3, 'beer' : 4, 'coke' : 5, 'S' : 6, 'E' : 7, '.' : 8}# 英语词表
idx2word = {i: w for i, w in enumerate(tgt_vocab)}# 构建英语词典，可实现通过id快速查找
tgt_vocab_size = len(tgt_vocab)# 获取英语词表长度

src_len = 5
tgt_len = 6 # 设置德语和英语句子的长度
def make_data(sentences):
    enc_inputs, dec_inputs, dec_outputs = [], [], [] # 构建好三个序列（对应数据集）
    for i in range(len(sentences)):# 遍历输入的每一行样本
      enc_input = [[src_vocab[n] for n in sentences[i][0].split()]] # [[1, 2, 3, 4, 0], [1, 2, 3, 5, 0]]，以每个空格为一个拆分点，拆成一个个词，前面的向量其实就是这些词对应的索引，依据词典
      dec_input = [[tgt_vocab[n] for n in sentences[i][1].split()]] # [[6, 1, 2, 3, 4, 8], [6, 1, 2, 3, 5, 8]]，以此类推
      dec_output = [[tgt_vocab[n] for n in sentences[i][2].split()]] # [[1, 2, 3, 4, 8, 7], [1, 2, 3, 5, 8, 7]]，以此类推

      enc_inputs.extend(enc_input)# 将编码后的样本分别添加到构建好的三个序列中
      dec_inputs.extend(dec_input)
      dec_outputs.extend(dec_output)

    return torch.LongTensor(enc_inputs), torch.LongTensor(dec_inputs), torch.LongTensor(dec_outputs)# 返回三个序列

enc_inputs, dec_inputs, dec_outputs = make_data(sentences)# 输入样本，开始上述操作进行对应编码！

class MyDataSet(Data.Dataset):# 创建编码数据集
  def __init__(self, enc_inputs, dec_inputs, dec_outputs):# 构造函数
    super(MyDataSet, self).__init__()
    self.enc_inputs = enc_inputs
    self.dec_inputs = dec_inputs
    self.dec_outputs = dec_outputs

  def __len__(self):
    return self.enc_inputs.shape[0]# 返回德语长度

  def __getitem__(self, idx):# 调用索引时自动执行，一次返回一行数据
    return self.enc_inputs[idx], self.dec_inputs[idx], self.dec_outputs[idx]

loader = Data.DataLoader(MyDataSet(enc_inputs, dec_inputs, dec_outputs), 2, True)# 一个编码数据集创建好了
for batch_idx, (enc_inputs, dec_inputs, dec_outputs) in enumerate(loader):# 输出编码好的数据集
    print(f"Batch {batch_idx + 1}:")
    print("Encoder Inputs:", enc_inputs)
    print("Decoder Inputs:", dec_inputs)
    print("Decoder Outputs:", dec_outputs)
    print("-" * 50)

Batch 1:
Encoder Inputs: tensor([[1, 2, 3, 4, 0],
        [1, 2, 3, 5, 0]])
Decoder Inputs: tensor([[6, 1, 2, 3, 4, 8],
        [6, 1, 2, 3, 5, 8]])
Decoder Outputs: tensor([[1, 2, 3, 4, 8, 7],
        [1, 2, 3, 5, 8, 7]])
--------------------------------------------------


上面的操作实现了样本的Embedding操作

# 2. 模型参数
下面变量代表的含义依次是

1. 字嵌入 & 位置嵌入的维度，这俩值是相同的，因此用一个变量就行了
2. FeedForward 层隐藏神经元个数
3. Q、K、V 向量的维度，其中 Q 与 K 的维度必须相等，V 的维度没有限制，不过为了方便起见，我都设为 64
4. Encoder 和 Decoder 的个数
5. 多头注意力中 head 的数量



In [4]:
# Transformer Parameters
d_model = 512  # Embedding Size
d_ff = 2048 # FeedForward dimension
d_k = d_v = 64  # dimension of K(=Q), V
n_layers = 6  # number of Encoder of Decoder Layer
n_heads = 8  # number of heads in Multi-Head Attention

# 3. 位置编码
这里主要是实现代入公式，公式就是那个三角函数的


In [9]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):# 对于模型维度等参数的设置
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)# 初始化为全0张量
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)#这几行都是实现公式
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)# 代入公式，偶数sin
        pe[:, 1::2] = torch.cos(position * div_term)# 奇数cos
        pe = pe.unsqueeze(0).transpose(0, 1)# 便于计算的操作
        self.register_buffer('pe', pe)

    def forward(self, x):# 前向传播，实现Embedding与位置编码结果相加
        '''
        x: [seq_len, batch_size, d_model]
        '''
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)



# 4.Pad Mask（掩码）
由于在 Encoder 和 Decoder 中都需要进行 mask 操作，因此就无法确定这个函数的参数中 seq_len 的值，如果是在 Encoder 中调用的，seq_len 就等于 src_len；如果是在 Decoder 中调用的，seq_len 就有可能等于 src_len，也有可能等于 tgt_len（因为 Decoder 有两次 mask）

这个函数最核心的一句代码是 seq_k.data.eq(0)，这句的作用是返回一个大小和 seq_k 一样的 tensor，只不过里面的值只有 True 和 False。如果 seq_k 某个位置的值等于 0，那么对应位置就是 True，否则即为 False。举个例子，输入为 seq_data = [1, 2, 3, 4, 0]，seq_data.data.eq(0) 就会返回 [False, False, False, False, True]

剩下的代码主要是扩展维度，强烈建议读者打印出来，看看最终返回的数据是什么样子

Pad Mask 的作用是确保模型不会在处理输入序列时关注到填充（padding）部分的内容。填充通常用于将不同长度的输入序列对齐为相同的长度，以便可以批量处理。但填充部分不包含有效的输入数据，因此模型不应关注这些填充部分，以免对最终输出产生不良影响。

我的理解是，通过返回True返回填充部分，识别到填充位置（特殊标记P）

In [10]:
def get_attn_pad_mask(seq_q, seq_k):
    '''
    seq_q: [batch_size, seq_len]
    seq_k: [batch_size, seq_len]
    seq_len could be src_len or it could be tgt_len
    seq_len in seq_q and seq_len in seq_k maybe not equal
    '''
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    # eq(zero) is PAD token
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # [batch_size, 1, len_k], True is masked
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # [batch_size, len_q, len_k]

# 5.Subsequence Mask
Subsequence Mask 只有 Decoder 会用到，主要作用是屏蔽未来时刻单词的信息。不让模型知道答案，这样会有效缓解过拟合的发生。首先通过 np.ones() 生成一个全 1 的方阵，然后通过 np.triu() 生成一个上三角矩阵

In [11]:
def get_attn_subsequence_mask(seq):
    '''
    seq: [batch_size, tgt_len]
    '''
    attn_shape = [seq.size(0), seq.size(1), seq.size(1)]
    subsequence_mask = np.triu(np.ones(attn_shape), k=1) # Upper triangular matrix(上三角矩阵)
    subsequence_mask = torch.from_numpy(subsequence_mask).byte()
    return subsequence_mask # [batch_size, tgt_len, tgt_len]

# 6.ScaledDotProductAttention
这一部分就是进行注意力分数的计算，套公式的事情

注意力机制原理：我往简单说就是比较Q与K的相似度（因此采用点积形式，比较两个向量重合了多少），下文中的attn就是Q中的一个元素与K中每个元素比较时，K的一个元素对于Q中该元素的影响权重（注意力分数），context是这个分数乘了该K对应的值V，就是得到的对应的注意力组成内容，这些一个个小小的注意力组成内容组成了一个综合了所有量的一个综合了比较量的所有信息的注意力张量，这个注意力分数也可以说是注意力权重


In [12]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        '''
        Q: [batch_size, n_heads, len_q, d_k]
        K: [batch_size, n_heads, len_k, d_k]
        V: [batch_size, n_heads, len_v(=len_k), d_v]
        attn_mask: [batch_size, n_heads, seq_len, seq_len]
        '''
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k) # 套公式计算
        scores.masked_fill_(attn_mask, -1e9) # 进行一个小的填充，因为掩码操作，填充是为了计算准确吧

        attn = nn.Softmax(dim=-1)(scores)# 归一化操作
        context = torch.matmul(attn, V) # [batch_size, n_heads, len_q, d_v]
        return context, attn # 返回注意力组成内容和注意力分数

# 7.MultiHeadAttention
多头注意力


In [13]:
class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.W_Q = nn.Linear(d_model, d_k * n_heads, bias=False)#设置 Q K V参数
        self.W_K = nn.Linear(d_model, d_k * n_heads, bias=False)
        self.W_V = nn.Linear(d_model, d_v * n_heads, bias=False)
        self.fc = nn.Linear(n_heads * d_v, d_model, bias=False)
    def forward(self, input_Q, input_K, input_V, attn_mask):
        '''
        input_Q: [batch_size, len_q, d_model]
        input_K: [batch_size, len_k, d_model]
        input_V: [batch_size, len_v(=len_k), d_model]
        attn_mask: [batch_size, seq_len, seq_len]
        '''
        residual, batch_size = input_Q, input_Q.size(0)
        # (B, S, D) -proj-> (B, S, D_new) -split-> (B, S, H, W) -trans-> (B, H, S, W)
        Q = self.W_Q(input_Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # Q: [batch_size, n_heads, len_q, d_k]# 对q进行变换操作，便于计算
        K = self.W_K(input_K).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # K: [batch_size, n_heads, len_k, d_k]
        V = self.W_V(input_V).view(batch_size, -1, n_heads, d_v).transpose(1,2)  # V: [batch_size, n_heads, len_v(=len_k), d_v]

        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1) # attn_mask : [batch_size, n_heads, seq_len, seq_len]这段代码主要是对 attn_mask 进行维度扩展和重复，以适应多头注意力机制的需求。

        # context: [batch_size, n_heads, len_q, d_v], attn: [batch_size, n_heads, len_q, len_k]
        context, attn = ScaledDotProductAttention()(Q, K, V, attn_mask)# 计算注意力分数和注意力内容
        context = context.transpose(1, 2).reshape(batch_size, -1, n_heads * d_v) # context: [batch_size, len_q, n_heads * d_v]方便计算
        output = self.fc(context) # [batch_size, len_q, d_model]对于注意力内容输出进行变换操作
        return nn.LayerNorm(d_model).cuda()(output + residual), attn # 进行层归一化操作，层归一化是对一个样本的每个特征进行归一化，具体来说，它对每个样本的每一层的所有特征维度进行标准化，提高训练稳定性

    # residual：是一个残差连接。残差就是把输出与输入相加，可以让模型变得复杂，利于模型训练，让它训练更快，且输入值input_Q经过线性变换趋向于0，模型还是等效的

# 8.FeedForward Layer
残差连接，这段代码非常简单，就是做两次线性变换，残差连接后再跟一个 Layer Norm

In [14]:
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(d_model, d_ff, bias=False),
            nn.ReLU(),
            nn.Linear(d_ff, d_model, bias=False)
        )
    def forward(self, inputs):
        '''
        inputs: [batch_size, seq_len, d_model]
        '''
        residual = inputs
        output = self.fc(inputs)
        return nn.LayerNorm(d_model).cuda()(output + residual)
    # residual：是一个残差连接。残差就是把输出与输入相加，可以让模型变得复杂，利于模型训练，让它训练更快，且输入值input_Q经过线性变换趋向于0，模型还是等效的（再次重复）

# 9.Encoder Layer
将上述组件拼起来，就是一个完整的 Encoder Layer

In [15]:
class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()# 计算注意力分数
        self.pos_ffn = PoswiseFeedForwardNet()# 残差连接

    def forward(self, enc_inputs, enc_self_attn_mask):
        '''
        enc_inputs: [batch_size, src_len, d_model]
        enc_self_attn_mask: [batch_size, src_len, src_len]
        '''
        # enc_outputs: [batch_size, src_len, d_model], attn: [batch_size, n_heads, src_len, src_len]
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # 相同的Q K V，因为是自注意力机制，得出每个元素综合自己序列所有信息的注意力内容和注意力分数（返回值的对应赋值）
        enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size, src_len, d_model]，注意力内容继续做残差连接
        return enc_outputs, attn # 返回注意力内容和注意力分数

# 10. Encoder
使用 nn.ModuleList() 里面的参数是列表，列表里面存了 n_layers 个 Encoder Layer

由于我们控制好了 Encoder Layer 的输入和输出维度相同，所以可以直接用个 for 循环以嵌套的方式，将上一次 Encoder Layer 的输出作为下一次 Encoder Layer 的输入

1个Encoder由许多Encoder Layer组成


In [16]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.src_emb = nn.Embedding(src_vocab_size, d_model)# Embedding编码
        self.pos_emb = PositionalEncoding(d_model)# 位置编码
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])# 存了 n_layers 个 Encoder Layer

    def forward(self, enc_inputs):# 前向传播返回结果
        '''
        enc_inputs: [batch_size, src_len]
        '''
        enc_outputs = self.src_emb(enc_inputs) # [batch_size, src_len, d_model]
        enc_outputs = self.pos_emb(enc_outputs.transpose(0, 1)).transpose(0, 1) # [batch_size, src_len, d_model]做个转化
        enc_self_attn_mask = get_attn_pad_mask(enc_inputs, enc_inputs) # [batch_size, src_len, src_len]Pad Mask 的作用是确保模型不会在处理输入序列时关注到填充（padding）部分的内容
        enc_self_attns = []
        for layer in self.layers:# 每一个小Encoder块
            # enc_outputs: [batch_size, src_len, d_model], enc_self_attn: [batch_size, n_heads, src_len, src_len]
            enc_outputs, enc_self_attn = layer(enc_outputs, enc_self_attn_mask)# 当前块的输出作为下一个块的输入
            enc_self_attns.append(enc_self_attn)# 添把注意力分数加到列表中
        return enc_outputs, enc_self_attns # 返回每一个块的注意力分数和总的输出

# 11. Decoder Layer
解码器生成序列

In [17]:
class DecoderLayer(nn.Module):
    def __init__(self):
        super(DecoderLayer, self).__init__()
        self.dec_self_attn = MultiHeadAttention()# 自注意力（对于输入的生成序列（这里是英语序列），先做一次自注意力机制，原因同Encoder）
        self.dec_enc_attn = MultiHeadAttention()# 注意力机制（生成新的序列时，要与上下文变量（即德语序列）进行注意力汇聚生成注意力内容，当然这里的英语序列是进行过自注意力机制的，汇聚了自己所有的信息的）
        self.pos_ffn = PoswiseFeedForwardNet()# 残差连接

    def forward(self, dec_inputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask):
        '''
        dec_inputs: [batch_size, tgt_len, d_model]
        enc_outputs: [batch_size, src_len, d_model]
        dec_self_attn_mask: [batch_size, tgt_len, tgt_len]
        dec_enc_attn_mask: [batch_size, tgt_len, src_len]
        '''
        # dec_outputs: [batch_size, tgt_len, d_model], dec_self_attn: [batch_size, n_heads, tgt_len, tgt_len]
        dec_outputs, dec_self_attn = self.dec_self_attn(dec_inputs, dec_inputs, dec_inputs, dec_self_attn_mask)# 先进行自注意力机制
        # dec_outputs: [batch_size, tgt_len, d_model], dec_enc_attn: [batch_size, h_heads, tgt_len, src_len]
        dec_outputs, dec_enc_attn = self.dec_enc_attn(dec_outputs, enc_outputs, enc_outputs, dec_enc_attn_mask)# 再进行注意力机制
        dec_outputs = self.pos_ffn(dec_outputs) # [batch_size, tgt_len, d_model]一个变换
        return dec_outputs, dec_self_attn, dec_enc_attn# 返回输出，自注意力分数，注意力分数

# 12. Decoder
结构同Encoder

In [18]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.tgt_emb = nn.Embedding(tgt_vocab_size, d_model)# 编码牢三件套
        self.pos_emb = PositionalEncoding(d_model)
        self.layers = nn.ModuleList([DecoderLayer() for _ in range(n_layers)])

    def forward(self, dec_inputs, enc_inputs, enc_outputs):# 原理同Encoder
        '''
        dec_inputs: [batch_size, tgt_len]
        enc_intpus: [batch_size, src_len]
        enc_outputs: [batch_size, src_len, d_model]
        '''
        dec_outputs = self.tgt_emb(dec_inputs) # [batch_size, tgt_len, d_model]
        dec_outputs = self.pos_emb(dec_outputs.transpose(0, 1)).transpose(0, 1).cuda() # [batch_size, tgt_len, d_model]Embedding和位置编码
        dec_self_attn_pad_mask = get_attn_pad_mask(dec_inputs, dec_inputs).cuda() # [batch_size, tgt_len, tgt_len]
        dec_self_attn_subsequence_mask = get_attn_subsequence_mask(dec_inputs).cuda() # [batch_size, tgt_len, tgt_len]
        dec_self_attn_mask = torch.gt((dec_self_attn_pad_mask + dec_self_attn_subsequence_mask), 0).cuda() # [batch_size, tgt_len, tgt_len]

        dec_enc_attn_mask = get_attn_pad_mask(dec_inputs, enc_inputs) # [batc_size, tgt_len, src_len] 生成注意力掩码

        dec_self_attns, dec_enc_attns = [], []
        for layer in self.layers:
            # dec_outputs: [batch_size, tgt_len, d_model], dec_self_attn: [batch_size, n_heads, tgt_len, tgt_len], dec_enc_attn: [batch_size, h_heads, tgt_len, src_len]
            dec_outputs, dec_self_attn, dec_enc_attn = layer(dec_outputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask)
            dec_self_attns.append(dec_self_attn)
            dec_enc_attns.append(dec_enc_attn)
        return dec_outputs, dec_self_attns, dec_enc_attns

Decoder 中不仅要把 "pad"mask 掉，还要 mask 未来时刻的信息，因此就有了下面这三行代码，其中 torch.gt(a, value) 的意思是，将 a 中各个位置上的元素和 value 比较，若大于 value，则该位置取 1，否则取 0
就是掩码机制，方便模型生成不看到答案，避免过拟合现象


In [19]:
dec_self_attn_pad_mask = get_attn_pad_mask(dec_inputs, dec_inputs) # [batch_size, tgt_len, tgt_len]
dec_self_attn_subsequence_mask = get_attn_subsequence_mask(dec_inputs) # [batch_size, tgt_len, tgt_len]
dec_self_attn_mask = torch.gt((dec_self_attn_pad_mask + dec_self_attn_subsequence_mask), 0) # [batch_size, tgt_len, tgt_len]

# Transformer

In [20]:
class Transformer(nn.Module):
    def __init__(self):
        super(Transformer, self).__init__()# 熟悉不
        self.encoder = Encoder().cuda()
        self.decoder = Decoder().cuda()
        self.projection = nn.Linear(d_model, tgt_vocab_size, bias=False).cuda()
    def forward(self, enc_inputs, dec_inputs):
        '''
        enc_inputs: [batch_size, src_len]
        dec_inputs: [batch_size, tgt_len]
        '''
        # tensor to store decoder outputs
        # outputs = torch.zeros(batch_size, tgt_len, tgt_vocab_size).to(self.device)

        # enc_outputs: [batch_size, src_len, d_model], enc_self_attns: [n_layers, batch_size, n_heads, src_len, src_len]
        enc_outputs, enc_self_attns = self.encoder(enc_inputs)# 德语序列经过编码器
        # dec_outpus: [batch_size, tgt_len, d_model], dec_self_attns: [n_layers, batch_size, n_heads, tgt_len, tgt_len], dec_enc_attn: [n_layers, batch_size, tgt_len, src_len]
        dec_outputs, dec_self_attns, dec_enc_attns = self.decoder(dec_inputs, enc_inputs, enc_outputs)#经过Encoder编码的德语序列、初始德语序列和英语序列进入解码器，初始德语序列用于生成注意力掩码
        dec_logits = self.projection(dec_outputs) # dec_logits: [batch_size, tgt_len, tgt_vocab_size]经过一个线性层
        return dec_logits.view(-1, dec_logits.size(-1)), enc_self_attns, dec_self_attns, dec_enc_attns# 方便计算的操作

# 模型 & 损失函数 & 优化器

In [21]:
model = Transformer().cuda()
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.99)

# 训练

In [22]:
for epoch in range(30):
    for enc_inputs, dec_inputs, dec_outputs in loader:
      '''
      enc_inputs: [batch_size, src_len]
      dec_inputs: [batch_size, tgt_len]
      dec_outputs: [batch_size, tgt_len]
      '''
      enc_inputs, dec_inputs, dec_outputs = enc_inputs.cuda(), dec_inputs.cuda(), dec_outputs.cuda()
      # outputs: [batch_size * tgt_len, tgt_vocab_size]
      outputs, enc_self_attns, dec_self_attns, dec_enc_attns = model(enc_inputs, dec_inputs)# 前向传播
      loss = criterion(outputs, dec_outputs.view(-1))# 损失
      print('Epoch:', '%04d' % (epoch + 1), 'loss =', '{:.6f}'.format(loss))

      optimizer.zero_grad()# 梯度清零
      loss.backward()# 反向传播
      optimizer.step()# 参数更新

Epoch: 0001 loss = 2.246585
Epoch: 0002 loss = 2.021879
Epoch: 0003 loss = 1.952262
Epoch: 0004 loss = 1.757086
Epoch: 0005 loss = 1.477152
Epoch: 0006 loss = 1.270006
Epoch: 0007 loss = 1.086807
Epoch: 0008 loss = 0.893156
Epoch: 0009 loss = 0.631914
Epoch: 0010 loss = 0.470819
Epoch: 0011 loss = 0.318672
Epoch: 0012 loss = 0.262762
Epoch: 0013 loss = 0.206329
Epoch: 0014 loss = 0.152529
Epoch: 0015 loss = 0.127350
Epoch: 0016 loss = 0.101656
Epoch: 0017 loss = 0.103941
Epoch: 0018 loss = 0.080807
Epoch: 0019 loss = 0.053581
Epoch: 0020 loss = 0.053457
Epoch: 0021 loss = 0.040541
Epoch: 0022 loss = 0.033012
Epoch: 0023 loss = 0.033353
Epoch: 0024 loss = 0.023781
Epoch: 0025 loss = 0.021974
Epoch: 0026 loss = 0.026839
Epoch: 0027 loss = 0.022746
Epoch: 0028 loss = 0.025649
Epoch: 0029 loss = 0.025477
Epoch: 0030 loss = 0.019975


# 测试

In [23]:
def greedy_decoder(model, enc_input, start_symbol):# 贪婪解码器（Greedy Decoder）通过逐字生成目标序列，始终选择当前最可能的单词（最大概率的单词）作为下一个单词，并将其加入到解码器的输入中，直到遇到结束符号（tgt_vocab["."]）。
    """
   为简单起见，当 K=1 时，贪婪解码器是 Beam 搜索。这对于推理是必要的，因为我们不知道
目标序列输入。因此我们尝试逐字生成目标输入，然后将其输入到转换器中。
起始参考：http://nlp.seas.harvard.edu/2018/04/03/attention.html#greedy-decoding
:param model: Transformer 模型
:param enc_input: 编码器输入
:param start_symbol: 起始符号。在此示例中，它是 'S'，对应于索引 4
:return: 目标输入
    """
    enc_outputs, enc_self_attns = model.encoder(enc_input)# 编码
    dec_input = torch.zeros(1, 0).type_as(enc_input.data)# 初始化全0矩阵
    terminal = False
    next_symbol = start_symbol
    while not terminal:
        dec_input = torch.cat([dec_input.detach(),torch.tensor([[next_symbol]],dtype=enc_input.dtype).cuda()],-1)
        dec_outputs, _, _ = model.decoder(dec_input, enc_input, enc_outputs)# 获取decoder输出
        projected = model.projection(dec_outputs)# 输出经过线性层
        prob = projected.squeeze(0).max(dim=-1, keepdim=False)[1]# 获取最大值的索引（decoder生成序列，每个单词对应一个概率序列，概率值最大的那个就是当前预测出的单词）
        next_word = prob.data[-1]# 为当前生成的单词的索引
        next_symbol = next_word# 下一个元素
        if next_symbol == tgt_vocab["."]:# 是否是句号
            terminal = True# 翻译结束
        print(next_word)# 输出那个单词
    return dec_input
# 这里再次重申，注意力分数（权重）——这个元素的重要性占比（我该在这个元素上投入多少注意力）
# 注意力内容——注意力分数与对应元素的乘积
# Test
enc_inputs, _, _ = next(iter(loader))
enc_inputs = enc_inputs.cuda()
for i in range(len(enc_inputs)):
    greedy_dec_input = greedy_decoder(model, enc_inputs[i].view(1, -1), start_symbol=tgt_vocab["S"])# 解码
    predict, _, _, _ = model(enc_inputs[i].view(1, -1), greedy_dec_input)
    predict = predict.data.max(1, keepdim=True)[1]# 生成每个单词得到索引
    print(enc_inputs[i], '->', [idx2word[n.item()] for n in predict.squeeze()])# 这几行是生成单词的，当把一句话的所有单词解码完后再生成一句话
    # 最后一行代码，通过索引加字典生成单词

tensor(1, device='cuda:0')
tensor(2, device='cuda:0')
tensor(3, device='cuda:0')
tensor(4, device='cuda:0')
tensor(8, device='cuda:0')
tensor([1, 2, 3, 4, 0], device='cuda:0') -> ['i', 'want', 'a', 'beer', '.']
tensor(1, device='cuda:0')
tensor(2, device='cuda:0')
tensor(3, device='cuda:0')
tensor(5, device='cuda:0')
tensor(8, device='cuda:0')
tensor([1, 2, 3, 5, 0], device='cuda:0') -> ['i', 'want', 'a', 'coke', '.']
